In [ ]:
import gurobipy as grb
import pandas as pd
import numpy as np
import scipy.sparse as sparse
import os

In [ ]:
thepath = os.getcwd().split("veteran_students_mec_optim\\Pauline")[0]

data_X = pd.read_csv(thepath + "\\data_mec_optim\\marriage_personality-traits\\Xvals.csv", sep=',')
data_Y = pd.read_csv(thepath + "\\data_mec_optim\\marriage_personality-traits\\Yvals.csv", sep=',')
affdf = pd.read_csv(thepath + "\\data_mec_optim\\marriage_personality-traits\\Affinitymatrix.csv", sep=',')

In [ ]:
data_X.head()

In [ ]:
nbcar = 10
affmat = affdf.iloc[0:nbcar,1:nbcar+1].values
affmat

In [ ]:
sdX = data_X.std().values
sdY = data_Y.std().values
mX = data_X.mean().values
mY = data_Y.mean().values

Xvals = (data_X-mX)/sdX
Yvals = (data_Y-mY)/sdY

In [ ]:
nobs = Xvals.shape[0]
Phi = Xvals.values.dot(affmat).dot(Yvals.values.T).T
c= Phi.flatten()

In [ ]:
c.shape
#len(coupleList)

In [ ]:
coupleList = [(i, j) for i in range(nobs) for j in range(nobs)]
m = grb.Model('marriage')
couple = m.addVars(coupleList, obj=c, name='couple')
m.ModelSense = -1

In [ ]:
m.addConstrs((couple.sum('*', women) == 1 for women in range(nobs)))
m.addConstrs((couple.sum(men, '*') == 1 for men in range(nobs)))

In [ ]:
m.optimize()

In [ ]:
if m.status == grb.GRB.Status.OPTIMAL:
    print("Value of the problem (Gurobi) =", m.objval)
    pi = m.getAttr(grb.GRB.Attr.Pi)
    u = pi[:nobs]
    v = pi[nobs:]
    print(u[:10])
    print("")
    print(v[:10])